# eEDB-011-2024-3

## Atividade 4 – Ingestão e ETL com linguagem de programação (Python + Spark + SQL)

* Utilizar linguagem de programação Python para ingestão e tratamento de dados. Para processo de transformação
deve ser realizado via SQL com uma das seguintes ferramentas:
  * DBT (indicada)
  *  SQL Mash – alternativa
  *  Coalesce - Alternativa

*  Base final deve ser um banco de dados relacional já utilizado nos exercícios anteriores.
* O processamento SQL deve ser utilizado via ferramenta mais uma engine de SQL que pode ser o próprio Banco de
Dados bem como um banco de dados em memória como Duckdb ou SLQLite, ou qualquer outra alternativa.
*  Gerar uma tabela final com os dados tratados e unidos.
  * O tratamento de dados deve ser realizado através da ferramenta escolhida para SQL
*  Adicionar as seguintes camadas de processamento, dentro do próprio banco de dados ou em disco local. A
Camada Delivery deve obrigatoriamente ter estar também no formato de uma tabela final dentro do banco de
dados relacional:
  *  RAW – formato dos dados livre
  *  Trusted – formato de dados em Parquet ou ORC or AVRO (indicado Parquet)
  *  Delivery– formato de dados em Parquet ou ORC or AVRO (indicado Parquet)

- **Grupo 02**:
    - Aline Bini
    - Ana Lívia Franco
    - Ana Priss
    - João Squinelato
    - Marcelo Pena
    - Thais Siqueira

- [Github](https://github.com/Squinelato/eEDB-011-2024-3 "eEDB-011-2024-3")

```Ingestão De Dados | Agosto 2024```

## To Do

- raw
- trusted
- delivery

In [3]:
%pip install pyspark
%pip install unidecode
%pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=b832e978e3d251e8ddf19a0dd263635298dc6b1168951a1c7b42f1c6d9742636
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lpad, col, lpad, concat, sha1, regexp_replace, udf, lower, lit, when
from pyspark.sql.types import StringType, FloatType, IntegerType, StructType, StructField, ArrayType
from unidecode import unidecode

import findspark
import os

In [5]:
findspark.init()

In [6]:
spark = SparkSession.builder \
    .master('local') \
    .appName('Basic ETL') \
    .config('spark.executor.timeout', "1200s") \
    .config('spark.sql.broadcastTimeout', '1200s') \
    .config('spark.rpc.askTimeout', '600s') \
    .config('spark.executor.heartbeatInterval', '120s') \
    .config('spark.network.timeout', '1200s') \
    .getOrCreate()

In [7]:
spark

---
## **Raw**

### **Banks file**

In [26]:
# banks_csv_path = '../Fonte de Dados/Bancos/EnquadramentoInicia_v2.tsv'
banks_csv_path = '/EnquadramentoInicia_v2.tsv'

# Leitura do arquivo CSV e criação da view temporária
rwzd_bank = spark.read.csv(banks_csv_path, sep='\t', encoding='utf8', header=True)
rwzd_bank.createOrReplaceTempView("rwzd_bank")
spark.sql("SELECT * FROM rwzd_bank LIMIT 100").show(truncate=False)

+--------+--------+---------------------------------------------+
|Segmento|CNPJ    |Nome                                         |
+--------+--------+---------------------------------------------+
|S1      |0       |BANCO DO BRASIL - PRUDENCIAL                 |
|S1      |60746948|BRADESCO - PRUDENCIAL                        |
|S1      |30306294|BTG PACTUAL - PRUDENCIAL                     |
|S1      |360305  |CAIXA ECONOMICA FEDERAL - PRUDENCIAL         |
|S1      |60872504|ITAU - PRUDENCIAL                            |
|S1      |90400888|SANTANDER - PRUDENCIAL                       |
|S2      |92702067|BANRISUL - PRUDENCIAL                        |
|S2      |7237373 |BANCO DO NORDESTE DO BRASIL S.A. - PRUDENCIAL|
|S2      |33657248|BNDES - PRUDENCIAL                           |
|S2      |33479023|CITIBANK - PRUDENCIAL                        |
|S2      |33987793|CREDIT SUISSE - PRUDENCIAL                   |
|S2      |58160789|SAFRA - PRUDENCIAL                           |
|S2      |

Analisando o esquema dos dados

In [28]:
spark.sql("DESCRIBE rwzd_bank").show(truncate=False)

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|Segmento|string   |NULL   |
|CNPJ    |string   |NULL   |
|Nome    |string   |NULL   |
+--------+---------+-------+



Contando a quantidade de linhas

In [17]:
total_records = spark.sql("SELECT COUNT(*) AS total FROM rwzd_bank").collect()[0]['total']
print(f"Total de registros: {total_records}")

Total de registros: 1474


Salvando dados na camada _raw_ no formato parquet

In [ ]:
# raw_bank_path = './raw/bank/'
# rwzd_bank.write.parquet(raw_bank_path, mode="append")

### **Employees file**

Localizando todos os arquivos contendo dados de empregados

In [29]:
# employee_dir = '../Fonte de Dados/Empregados/'
employee_dir = '/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados'
employee_files = os.listdir(employee_dir)
employee_paths = list(map(lambda file: os.path.join(employee_dir, file), employee_files))[1]
employee_paths

'/content/drive/MyDrive/eEDB-011-2024-3/Fonte de Dados/Bancos'

Lendo todos os arquivos de empregados como um único conjunto de dados

In [30]:
rwzd_employee = spark.read.csv(employee_paths, sep='\t', encoding='utf8', header=True)
rwzd_employee.createOrReplaceTempView("rwzd_employee")
spark.sql("SELECT * FROM rwzd_employee LIMIT 100").show()

+--------+--------+--------------------+
|Segmento|    CNPJ|                Nome|
+--------+--------+--------------------+
|      S1|       0|BANCO DO BRASIL -...|
|      S1|60746948|BRADESCO - PRUDEN...|
|      S1|30306294|BTG PACTUAL - PRU...|
|      S1|  360305|CAIXA ECONOMICA F...|
|      S1|60872504|   ITAU - PRUDENCIAL|
|      S1|90400888|SANTANDER - PRUDE...|
|      S2|92702067|BANRISUL - PRUDEN...|
|      S2| 7237373|BANCO DO NORDESTE...|
|      S2|33657248|  BNDES - PRUDENCIAL|
|      S2|33479023|CITIBANK - PRUDEN...|
|      S2|33987793|CREDIT SUISSE - P...|
|      S2|58160789|  SAFRA - PRUDENCIAL|
|      S2|59588111|VOTORANTIM - PRUD...|
|      S3|28195667|ABC-BRASIL - PRUD...|
|      S3|60770336|   ALFA - PRUDENCIAL|
|      S3|  655522|          APE POUPEX|
|      S3| 2992446|BANCO CNH INDUSTR...|
|      S3| 2038232|BANCOOB - PRUDENCIAL|
|      S3|28127603|BANESTES - PRUDEN...|
|      S3|31597552|BANCO CLASSICO S....|
+--------+--------+--------------------+
only showing top

Removendo duplicatas com base no nome e segmento do banco

In [ ]:
# Remover duplicatas com base nas colunas 'Nome' e 'Segmento'
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW rwzd_employee_dedup AS
SELECT *
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY Nome, Segmento ORDER BY Nome) AS row_num
    FROM rwzd_employee
) tmp
WHERE row_num = 1
""")
spark.sql("SELECT * FROM rwzd_employee_dedup").show(truncate=False)

Analisando o esquema dos dados

In [ ]:
rwzd_employee.printSchema()

root
 |-- employer_name: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- culture_count: string (nullable = true)
 |-- salaries_count: string (nullable = true)
 |-- benefits_count: string (nullable = true)
 |-- employer-website: string (nullable = true)
 |-- employer-headquarters: string (nullable = true)
 |-- employer-founded: string (nullable = true)
 |-- employer-industry: string (nullable = true)
 |-- employer-revenue: string (nullable = true)
 |-- url: string (nullable = true)
 |-- Geral: string (nullable = true)
 |-- Cultura e valores: string (nullable = true)
 |-- Diversidade e inclusão: string (nullable = true)
 |-- Qualidade de vida: string (nullable = true)
 |-- Alta liderança: string (nullable = true)
 |-- Remuneração e benefícios: string (nullable = true)
 |-- Oportunidades de carreira: string (nullable = true)
 |-- Recomendam para outras pessoas(%): string (nullable = true)
 |-- Perspectiva positiva da empresa(%): string (nullable = true)
 |-- Seg

Contando a quantidade de linhas

In [ ]:
rwzd_employee.count()

32

Salvando dados na camada _raw_ no formato parquet

In [ ]:
# raw_employee_path = './raw/employee/'
# rwzd_employee.write.parquet(raw_employee_path, mode="append")

### **Claims**

Localizando todos os arquivos contendo dados de reclamações

In [ ]:
claim_dir = '../Fonte de Dados/Reclamações/'
claim_files = os.listdir(claim_dir)
claim_paths = list(map(lambda file: os.path.join(claim_dir, file), claim_files))
claim_paths

['../Fonte de Dados/Reclamações/2021_tri_01.csv',
 '../Fonte de Dados/Reclamações/2021_tri_02.csv',
 '../Fonte de Dados/Reclamações/2021_tri_03.csv',
 '../Fonte de Dados/Reclamações/2021_tri_04.csv',
 '../Fonte de Dados/Reclamações/2022_tri_01.csv',
 '../Fonte de Dados/Reclamações/2022_tri_02_nao_ha_dados.csv',
 '../Fonte de Dados/Reclamações/2022_tri_03.csv',
 '../Fonte de Dados/Reclamações/2022_tri_04.csv']

In [ ]:
rwzd_claim = spark.read.csv(claim_paths, sep=';', encoding='latin1', header=True)
rwzd_claim.show()

+----+---------+--------------------+----------------+--------+----------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+----+
| Ano|Trimestre|           Categoria|            Tipo| CNPJ IF|Instituição financeira|Índice|Quantidade de reclamações reguladas procedentes|Quantidade de reclamações reguladas - outras|Quantidade de reclamações não reguladas|Quantidade total de reclamações|Quantidade total de clientes  CCS e SCR|Quantidade de clientes  CCS|Quantidade de clientes  SCR|_c14|
+----+---------+--------------------+----------------+--------+----------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------

Analisando o esquema dos dados

In [ ]:
rwzd_claim.printSchema()

root
 |-- Ano: string (nullable = true)
 |-- Trimestre: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- CNPJ IF: string (nullable = true)
 |-- Instituição financeira: string (nullable = true)
 |-- Índice: string (nullable = true)
 |-- Quantidade de reclamações reguladas procedentes: string (nullable = true)
 |-- Quantidade de reclamações reguladas - outras: string (nullable = true)
 |-- Quantidade de reclamações não reguladas: string (nullable = true)
 |-- Quantidade total de reclamações: string (nullable = true)
 |-- Quantidade total de clientes  CCS e SCR: string (nullable = true)
 |-- Quantidade de clientes  CCS: string (nullable = true)
 |-- Quantidade de clientes  SCR: string (nullable = true)
 |-- _c14: string (nullable = true)



Removendo coluna desnecessária

In [ ]:
rwzd_claim = rwzd_claim.drop('_c14')

Contando a quantidade de linhas

In [ ]:
rwzd_claim.count()

918

Salvando dados na camada _raw_ no formato parquet

In [ ]:
# raw_claim_path = './raw/claim/'
# rwzd_claim.write.parquet(raw_claim_path, mode="append")